# Exploratory ChatBotProject for Dementia Analysis
* Objective: Using Gemini model 2.0 flash - analyze if user entered diagnosis is categorized as one of the following:
    - Non Dementia
    - Very Mild Dementia
    - Mild Dementia
    - Moderate Dementia
* Use Gemini model 2.0 flash, make sure you have the Gemini API Key
* Use Gradio to show the result
* Disclaimer: This tool is assistive only, not to replace a doctor's or clinical diagnosis
* Example for some of the chabot answers are provided at the end of this Jupyter Notebook
    

In [1]:
# Import required libraries

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
import os

# Model and API Key

In [2]:
# Load environment variables from .env file
load_dotenv()

# Set up the Google Generative AI model for our LLMs
GEMINI_MODEL = "gemini-2.0-flash"

# Store the API Key in a variable
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

# Basic Query

In [3]:
# Initialize the model.
llm = ChatGoogleGenerativeAI(
    google_api_key=GEMINI_API_KEY,
    model=GEMINI_MODEL,
    temperature=0.2  # Adjust temperature for more or less randomness, lower is more deterministic, higher is more creative. For this project we will use 0.2
)

In [4]:
# Create a chain to classify dementia stages
prompt = PromptTemplate.from_template("""
    You are a dementia diagnostic assistant trained to classify cognitive impairment based on clinical descriptions.
    
    Classify the dementia stage based on the symptoms or health descriptions provided.
    
    Use these specific criteria for classification:
    
    - Non-dementia: No significant memory problems. May have occasional forgetfulness but no impact on daily activities. Fully independent in all activities of daily living. No concerning cognitive changes beyond normal aging.
    
    - Very mild dementia: Subtle but noticeable memory lapses (like misplacing objects or forgetting names). Minor word-finding difficulties. Still independent in daily activities but may take longer to complete complex tasks. Minimal impact on work or social activities.
    
    - Mild dementia: Clear memory deficits affecting recent events. Problems with time orientation or getting lost in familiar places. Difficulty with problem-solving and complex tasks. Beginning to need help with some instrumental activities of daily living (finances, transportation). Social withdrawal or mood changes may be present.
    
    - Moderate dementia: Significant memory loss including personal history. Disorientation to time and place. Impaired judgment and reasoning. Needs assistance with basic activities of daily living (dressing, bathing, etc.). May exhibit behavioral changes like agitation, suspicion, or wandering. Communication difficulties becoming apparent.
    
    Patient Symptoms or Health Data:
    {input_text}
    
    Analyze the information carefully and respond ONLY with one of these classifications:
    - Non-dementia
    - Very mild dementia
    - Mild dementia
    - Moderate dementia
    """)


In [5]:

def classify_dementia(input_text):
    final_prompt = prompt.format(input_text=input_text)
    result = llm.invoke(final_prompt)
    return result.content.strip()


In [6]:
# Test with sample inputs
example_1 = "Patient is healthy, no cognitive issues, and has no memory problems."
example_2 = "Patient has difficulty with complex tasks, often forgets names and faces, but can still live independently."
example_3 = "Patient forgets recent events, has trouble finding words, but still can manage daily tasks."
example_4 = "Patient requires assistance for dressing, frequently confused about time and place."


print("Example 1:", classify_dementia(example_1))
print("Example 2:", classify_dementia(example_2))
print("Example 3:", classify_dementia(example_3))
print("Example 4:", classify_dementia(example_4))


Example 1: - Non-dementia
Example 2: Very mild dementia
Example 3: Mild dementia
Example 4: Moderate dementia


# Use Gradio as to show the chatbot conversations

In [7]:
# Uncomment below if gradio is not installed
# !pip install gradio


In [8]:
import gradio as gr

def dementia_chatbot_interface(user_input):
    try:
        result = classify_dementia(user_input)
        return f"🧠 Dementia Classification: **{result}**"
    except Exception as e:
        return f"❌ Error: {str(e)}"

gr.Interface(
    fn=dementia_chatbot_interface,
    inputs=gr.Textbox(lines=5, placeholder="Enter patient's symptoms or health descriptions..."),
    outputs="markdown",
    title="🧠 Dementia Classification Chatbot",
    description="Enter symptoms or health data. The AI will classify the dementia stage.",
    allow_flagging="never",
    theme="default"
).launch(share=True)

/opt/anaconda3/envs/dev/lib/python3.12/site-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7866
* Running on public URL: https://93281dedeeb0cdfcdc.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Example entries
* Patient has a pacemaker, a smoker, 83 years old female --> Non dementia
* 22 year old female occasionally has brain fog, remembers all relationships with others, has clear recognition of her surroundings --> Non dementia
* Patient is a 69 years old female, sometimes misplace things --> Very mild dementia
* Patient is a 75 years old male, forgets his wife's birthday, sometimes get lost in the house --> Mild dementia
* Patient forgets recent events, has trouble finding words, 79 years old female --> Mild dementia
* Patient is a 75 years old male, forgets his wife's name and birthday, sometimes get lost in the house --> Moderate dementia
* Patient is a 77 years old male, an alcoholic, has had brain cancer, and always gets confused and lost --> Moderate dementia

